# Fine-tuning

Motivation:
* Adaptation of pre-trained Transformer models to a specific task
* Leveraging models trained on massive datasets
* Saving time and power

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Taking a look at the content of the Europarl-ST dataset having English as a source language

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")
print(raw_datasets)

The original dataset includes the translation from English into several European languages

In [3]:
raw_datasets["train"].features

{'source_text': Value(dtype='string', id=None),
 'dest_text': Value(dtype='string', id=None),
 'dest_lang': ClassLabel(names=['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ro'], id=None)}

Let us take a look at the translations of the first English sentence

In [4]:
raw_datasets["train"][:7]["source_text"]

['Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.']

In [ ]:
raw_datasets["train"][:7]["dest_text"]

['Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.',
 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.',
 'Depuis 1977, la plupart des services financiers, dont les assurances et la gestion des fonds de placement, ne sont pas tenus d ’ appliquer une TVA.',
 'Dal 1997 la maggior parte dei servizi finanziari, compresi i servizi assicurativi e la gestione di fondi di investimento, sono esenti da IVA.',
 'Sinds 1977 zijn de meeste financiële diensten, met inbegrip van verzekeringen en het beheer van beleggingsfondsen, vrijgesteld van btw.',
 'większość usług finansowych, w tym usług w zakresie ubezpieczeń i zarządzania funduszami inwestycyjnymi, była zwolniona z opodatkowania podatkiem VAT.',
 'Desde 1977 que a maioria dos serviços financeiros, incluindo os seguros e a gestão de fundos de investimento,

In [ ]:
raw_datasets["train"][:7]["dest_lang"]

[0, 2, 3, 4, 5, 6, 7]

Filtering Europarl-ST only for English into Spanish using a simple [lambda function](https://realpython.com/python-lambda/) with the [Dataset.filter() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.filter)

In [6]:
lang="es"
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id)

  0%|          | 0/603 [00:00<?, ?ba/s]

A good practice is to take a small random sample to get a quick feel for the type of data you’re working with. In Datasets, we can create a random sample by chaining the [Dataset.shuffle()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.shuffle) and [Dataset.select()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.select) functions together:

In [7]:
raw_sample = raw_datasets["train"].shuffle(seed=13).select(range(5))
raw_sample[:5]

{'source_text': ['the general issues of religious freedom, this is an absolute scandal and I welcome the fact that a number of Christian bishops from Iraq are to travel to Strasbourg in December',
  'The Commission fully supports the work of the International Criminal Tribunal for Yugoslavia, the ICTY.',
  'Another boost will come following the wonderful achievement of Swansea City Football Club – and I say this for the benefit of my colleague Richard Howitt – who have just deservedly reached the Premier Football League after a fantastic season.',
  'In conclusion, I would like to thank Parliament for its support and constructive approach.',
  'The European Council endorses the long-term vision for biodiversity by 2050 and the 2020 target set forth in the aforementioned Council conclusions of 15 March 2010.'],
 'dest_text': ['de los problemas de libertad religiosa, esto es un escándalo absoluto y celebro que varios obispos cristianos iraquíes vayan a viajar a Estrasburgo en diciembre',

Now we load the pre-trained tokenizer for the mT5 model and apply it to a sample English-Spanish pair

In [10]:
from transformers import AutoConfig, AutoTokenizer

checkpoint = "google/mt5-small"
config = AutoConfig.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
input_ids = tokenizer(raw_datasets["train"][0]["source_text"]).input_ids
print(input_ids)
decoder_input_ids = tokenizer(raw_datasets["train"][0]["dest_text"]).input_ids
print(decoder_input_ids)
decoder_input_ids = [config.decoder_start_token_id] + decoder_input_ids
print(decoder_input_ids)

[259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1]
[501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 31649, 260, 1]
[0, 501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 31649, 260, 1]


In [ ]:
print(tokenizer.convert_ids_to_tokens(decoder_input_ids))

['<pad>', '▁La', '▁mayor', 'ía', '▁de', '▁los', '▁', 'servicios', '▁financier', 'os', ',', '▁inclui', 'dos', '▁los', '▁', 'seguro', 's', '▁', 'y', '▁la', '▁', 'g', 'estión', '▁de', '▁fond', 'os', '▁de', '▁in', 'versión', ',', '▁está', 'n', '▁ex', 'ent', 'os', '▁de', '▁', 'IVA', '▁', 'desde', '▁1977', '.', '</s>']


In [ ]:
tokenizer.decode(decoder_input_ids)

'<pad> La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.</s>'

We can apply the tokenizer function to any dataset taking advantage that Hugging Face Datasets are [Apache Arrow](https://arrow.apache.org) files stored on the disk, so you only keep the samples you ask for loaded in memory.

To keep the data as a dataset, we will use the Dataset.map() method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset, so let us define a function that tokenizes our inputs:

In [56]:
def tokenize_function(sample):
    model_inputs = tokenizer(sample["source_text"])
    model_inputs['labels'] = tokenizer(sample["dest_text"]).input_ids
    #decoder_input_ids = [config.decoder_start_token_id] + decoder_input_ids
    return model_inputs

In [57]:
tokenized_sample = tokenize_function(raw_datasets["train"][0])
print(tokenized_sample)

{'input_ids': [259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 31649, 260, 1]}


The way the Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the tokenize function:

In [58]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/77 [00:00<?, ?ba/s]

  0%|          | 0/82 [00:00<?, ?ba/s]

  0%|          | 0/87 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 76469
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 81968
    })
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 86170
    })
})

The function that is responsible for putting together samples inside a batch is called a collate function. It is an argument you can pass when you build a DataLoader, the default being a function that will just convert your samples to PyTorch tensors and concatenate them. This is not possible in our case since the inputs we have are not all of the same size. We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding. 

To do this in practice, we have to define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together. Fortunately, the Transformers library provides us with such a function via DataCollatorWithPadding. It takes a tokenizer when you instantiate it (to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs) and will do everything you need:

In [59]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [60]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

To take a look at how this function works, now we have our dataset encoded, let us select only those keys that will be needed to create the batch discarding the textual features.

In [61]:
samples = [tokenized_datasets["train"][i] for i in range(2)]
print(samples)
for i in range(len(samples)):
    samples[i] = {key: value for key, value in samples[i].items() if key not in ["source_text", "dest_text", "dest_lang"]}
print(samples)

[{'source_text': 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.', 'dest_text': 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.', 'dest_lang': 2, 'input_ids': [259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 31649, 260, 1]}, {'source_text': 'During this period, two problems have essentially arisen: the definition of the scope of the exemption and the impossibility of recovering VAT incurred in order to provide exempt serv

In [ ]:
batch = data_collator(samples)
print(batch)

## Training

The first step before we can define our Trainer is to define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
model_name = checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-en-to-es",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
)

This is because mT5 has not been pretrained on translation, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a Trainer by passing it all the objects constructed up to now — the model, the training_args, the training and validation datasets, our data_collator, and our tokenizer:

In [71]:
from transformers import Seq2SeqTrainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


To fine-tune the model on our dataset, we just have to call the train() method of our Trainer:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: dest_text, source_text, dest_lang. If dest_text, source_text, dest_lang are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 76469
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 28677
  Number of trainable parameters = 300176768


Step,Training Loss
